# TEPEDELEN Léo DELLARICA Steven BARBIN Kevin

# Projet EISI1 Capgemini MSPRTPRE813_1

Ce notebook Python est le rapport détaillé et l'ETL de notre projet. La suite du rapport se situe après l'ETL, nous y détaillons le fonctionnement du reste du système.

Il a été réalisé à l'aide de Docker afin d'en faciliter le déploiement pour tester et collaborer.

Vous trouverez tout ce qu'il vous faut pour essayer le programme sur votre machine dans le fichier README de ce repository github :

# Extraire les données dans un DataFrame pySpark

In [2]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("example").getOrCreate()

# # Lire le fichier CSV
# df = spark.read.option("sep", "\t").csv("data/en.openfoodfacts.org.products.csv.gz", header=True, inferSchema=True).cache()

In [3]:
# print("Nombre de lignes dans le fichier de base : " + str(df.count()))

In [4]:
import pandas as pd

In [5]:
df_demographie = pd.read_csv('data/popcommunes.csv')

/tmp/ipykernel_1200/1901588191.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demographie = pd.read_csv('data/popcommunes.csv')


In [4]:
df_demographie.columns.to_list()

['dep',
 'nomdep',
 'codecommune',
 'nomcommune',
 'paris',
 'reg',
 'nomreg',
 'codeagglo',
 'nomagglo',
 'multicommune',
 'numcommune',
 'popagglo1780',
 'popagglo1781',
 'popagglo1782',
 'popagglo1783',
 'popagglo1784',
 'popagglo1785',
 'popagglo1786',
 'popagglo1787',
 'popagglo1788',
 'popagglo1789',
 'popagglo1790',
 'popagglo1791',
 'popagglo1792',
 'popagglo1793',
 'popagglo1794',
 'popagglo1795',
 'popagglo1796',
 'popagglo1797',
 'popagglo1798',
 'popagglo1799',
 'popagglo1800',
 'popagglo1801',
 'popagglo1802',
 'popagglo1803',
 'popagglo1804',
 'popagglo1805',
 'popagglo1806',
 'popagglo1807',
 'popagglo1808',
 'popagglo1809',
 'popagglo1810',
 'popagglo1811',
 'popagglo1812',
 'popagglo1813',
 'popagglo1814',
 'popagglo1815',
 'popagglo1816',
 'popagglo1817',
 'popagglo1818',
 'popagglo1819',
 'popagglo1820',
 'popagglo1821',
 'popagglo1822',
 'popagglo1823',
 'popagglo1824',
 'popagglo1825',
 'popagglo1826',
 'popagglo1827',
 'popagglo1828',
 'popagglo1829',
 'popagglo18

In [6]:
df_demographie_filtre = df_demographie.filter(regex="^dep$|^codecommune$|^reg$|^pop[0-9]{4}$")
df_demographie_filtre = spark.createDataFrame(df_demographie_filtre)

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1. Create the dimension table for commune data (df_dimension_commune) and add an index
window = Window.orderBy("dep", "codecommune", "reg")
df_dimension_commune = df_demographie_filtre.select("dep", "codecommune", "reg").distinct() \
    .withColumn("commune_id", F.row_number().over(window))

# 2. Extract population columns (columns starting with "pop" followed by 4 digits)
pop_columns = [col for col in df_demographie_filtre.columns if col.startswith('pop') and len(col) == 7 and col[3:].isdigit()]

# 3. Pivot the population columns into long format for the facts table (df_faits)
df_faits = df_demographie_filtre \
    .withColumn("year_population", F.explode(F.array(
        *[F.struct(F.lit(col[-4:]).alias("year"), F.col(col).alias("population")) for col in pop_columns]
    )))

# Split the "year_population" struct into separate columns for year and population
df_faits = df_faits.withColumn("year", F.col("year_population.year")) \
                   .withColumn("population", F.col("year_population.population")) \
                   .drop("year_population")

# 4. Join the facts table with the commune table to include commune_id in df_faits
df_faits = df_faits.join(df_dimension_commune, on=["dep", "codecommune", "reg"], how="inner") \
                   .select("commune_id", "year", "population")

# Display the results
df_dimension_commune.show()
df_faits.show()

+---+-----------+---+----------+
|dep|codecommune|reg|commune_id|
+---+-----------+---+----------+
|  1|       1001| 84|         1|
|  1|       1002| 84|         2|
|  1|       1004| 84|         3|
|  1|       1005| 84|         4|
|  1|       1006| 84|         5|
|  1|       1007| 84|         6|
|  1|       1008| 84|         7|
|  1|       1009| 84|         8|
|  1|       1010| 84|         9|
|  1|       1011| 84|        10|
|  1|       1012| 84|        11|
|  1|       1013| 84|        12|
|  1|       1014| 84|        13|
|  1|       1015| 84|        14|
|  1|       1016| 84|        15|
|  1|       1017| 84|        16|
|  1|       1019| 84|        17|
|  1|       1021| 84|        18|
|  1|       1022| 84|        19|
|  1|       1023| 84|        20|
+---+-----------+---+----------+
only showing top 20 rows

+----------+----+----------+
|commune_id|year|population|
+----------+----+----------+
|       150|1780|      1040|
|       150|1781|      1041|
|       150|1782|      1042|
|       

In [12]:
df_dimension_commune.count()

34856

In [6]:
df_faits = pd.DataFrame(columns=["population"])
df_faits = spark.createDataFrame(df_faits)

df_dimension_commune = pd.DataFrame(columns=["code_departement", "code_commune", "code_region"])
df_dimension_commune = spark.createDataFrame(df_dimension_commune)

df_dimension_annee = pd.DataFrame(columns=["annee"])
df_dimension_annee = spark.createDataFrame(df_dimension_annee)

for idx, row in df_demographie_filtre.iterrows():
    df_dimension_commune.loc[len(df_dimension_commune)] = [row["dep"], row["codecommune"], row["reg"]]
    for index, value in row.filter(regex="^pop[0-9]{4}$").items():
        df_faits.loc[len(df_faits)] = [value]
        df_dimension_annee.loc[len(df_dimension_annee)] = [index[-4:]]

KeyboardInterrupt: 

In [ ]:
df_faits

In [ ]:
df_dimensions

# Générer une table au format CSV avec une description statistique du DataFrame

In [3]:
# df.describe().toPandas().to_csv("describe_summary.csv", index=False)

# Effectuer un échantillonage du DataFrame pour en observer les colonnes qui peuvent sembler pertinentes

In [4]:
FRACTION_SIZE = 0.00001

In [5]:
columnsToSample = ['quantity', 'serving_size', 'serving_quantity', 'product_quantity']

In [6]:
samples_not_clean_df = df.select(columnsToSample).dropna(how='all').cache()

In [7]:
samples_df = samples_not_clean_df.sample(withReplacement=False, fraction=FRACTION_SIZE).cache()

In [8]:
samples_df.count()

16

In [9]:
samples_df.show()

+--------+-----------------+----------------+----------------+
|quantity|     serving_size|serving_quantity|product_quantity|
+--------+-----------------+----------------+----------------+
|    NULL|8.45 OZA (250 ml)|           250.0|            NULL|
|    NULL| 0.5 cup (120 ml)|           120.0|            NULL|
|    NULL|           340.0g|           340.0|            NULL|
|    NULL|    1 OZA (30 ml)|            30.0|            NULL|
|    NULL|   14 ONZ (396 g)|           396.0|            NULL|
|   270 g|             NULL|            NULL|           270.0|
|   100 g|             NULL|            NULL|           100.0|
|    300g|             NULL|            NULL|           300.0|
|   500 g|             NULL|            NULL|           500.0|
|   20 oz|             NULL|            NULL|     566.9904625|
|  2000 g|             NULL|            NULL|          2000.0|
|   100 g|       30 g (3 u)|            NULL|           100.0|
|   500 g|             NULL|            NULL|          

# Extraire les valeurs uniques d'une colonne afin d'en confirmer la pertinence (colonne ingredients_analysis_tags en exemple)

In [10]:
def flatten_list(li):
    flat_list = []
    for row in li:
        flat_list += row
    return flat_list

def make_list_unique(li):
    return list(dict.fromkeys(li))

def split_string_list_elements(li, sep):
    return [x.split(sep) for x in li]

def column_to_list(col):
    return col.rdd.flatMap(lambda x: x).collect()

In [11]:
ingredients_analysis_list = make_list_unique(flatten_list(split_string_list_elements(column_to_list(df.select('ingredients_analysis_tags').dropna()), ",")))

In [12]:
ingredients_analysis_list

['en:palm-oil-free',
 'en:non-vegan',
 'en:vegetarian-status-unknown',
 'en:palm-oil-content-unknown',
 'en:vegan-status-unknown',
 'en:may-contain-palm-oil',
 'en:vegetarian',
 'en:vegan',
 'en:non-vegetarian',
 'en:palm-oil',
 'en:maybe-vegan',
 'en:maybe-vegetarian']

# Sélection des colonnes nécessaires après étude de celles-ci

In [13]:
kept_columns = ["code", "product_name", "product_quantity", "energy-kcal_100g", "fat_100g", "saturated-fat_100g", "monounsaturated-fat_100g", "polyunsaturated-fat_100g", "trans-fat_100g", 
                "carbohydrates_100g", "sugars_100g", "starch_100g", "fiber_100g", "proteins_100g", "allergens", "traces", "vitamin-a_100g", "vitamin-c_100g", "vitamin-d_100g",
                "vitamin-e_100g", "vitamin-k_100g", "vitamin-b1_100g", "vitamin-b2_100g", "vitamin-b6_100g", "vitamin-b9_100g", "vitamin-b12_100g", "calcium_100g",
                "iron_100g", "magnesium_100g", "potassium_100g", "zinc_100g", "food_groups_tags", "serving_size", "serving_quantity", "cholesterol_100g", "salt_100g", "glycemic-index_100g"]

In [14]:
df_kept_columns = df.select(kept_columns)

# Qualité des données

## On supprime les lignes qui ont moins de 15 colonnes très pertinentes non nulles

In [15]:
columns_to_check = ["code", "product_name", "product_quantity", "energy-kcal_100g", "fat_100g", "saturated-fat_100g", "monounsaturated-fat_100g",
                    "polyunsaturated-fat_100g", "trans-fat_100g", "carbohydrates_100g", "sugars_100g", "starch_100g", 
                    "fiber_100g", "proteins_100g", "allergens", "traces", "food_groups_tags", "serving_size", "serving_quantity"]

In [16]:
df_kept_columns = df_kept_columns.dropna(subset=columns_to_check, thresh=15)

## On supprime les lignes dupliquées et on garde le DF en cache pour éviter de le recalculer

In [17]:
df_kept_columns = df_kept_columns.dropDuplicates().cache()

In [18]:
print("Nombre de lignes restantes après le traitement : " + str(df_kept_columns.count()))

Nombre de lignes restantes après le traitement : 65401


## Plus besoin du DataFrame de base donc on le retire de la mémoire cache

In [19]:
df.unpersist()
print("Mémoire libérée")

Mémoire libérée


# Écriture des données nettoyées en base de données avec le driver JDBC

In [20]:
properties = {
    "user": "user",
    "password": "userpassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}

df_kept_columns.write.jdbc(url="jdbc:mysql://mysql:3306/openfoodfact", table="products", mode="append", properties=properties)